## Imports

In [ ]:
import os
import pandas as pd
import sklearn.model_selection as sk
import gc
import transformers
import torch
import csv
import re
import subprocess
from IPython.display import FileLink

## Carga de datos y division del data set

In [ ]:
# Check if the zip file is present and has been unzipped
if not os.path.exists("cefr-levelled-english-texts.zip"):
    # Download the dataset if the zip file is not present
    !kaggle datasets download -d amontgomerie/cefr-levelled-english-texts

if not os.path.exists("cefr_leveled_texts.csv"):  # Adjust this to match the folder name after unzipping
    # Unzip the file if the unzipped folder does not exist
    !unzip cefr-levelled-english-texts.zip

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('cefr_leveled_texts.csv')

# Get the minimum number of samples in any class
min_samples = df['label'].value_counts().min()

# Downsample each class to have the same number of samples as the smallest class
df_balanced = df.groupby('label').apply(lambda x: x.sample(n=min_samples, random_state=60)).reset_index(drop=True)

distribution = df_balanced['label'].value_counts()
train, div = sk.train_test_split(df_balanced, test_size=0.2, random_state=70)
dev, holdout = sk.train_test_split(div, test_size=0.5, random_state=50)
#holdout, dev = sk.train_test_split(div, test_size=0.08, random_state=50)

train = train.reset_index(drop=True)
div = div.reset_index(drop=True)
holdout = holdout.reset_index(drop=True)

textos_metricas = pd.concat([dev, train])

## Carga del modelo

In [ ]:
model_name = "tareknaous/readabert-en"

device = "cuda"

classifier = transformers.pipeline(
  "text-classification",
  model=model_name,
  truncation=True,
  device=device  
)

## Ejecucion del experimento

In [ ]:
label_mapping = {
    'LABEL_1': 'A1',
    'LABEL_2': 'A2',
    'LABEL_3': 'B1',
    'LABEL_4': 'B2',
    'LABEL_5': 'C1',
    'LABEL_6': 'C2'
}

In [ ]:
gc.collect()

filename =f'experimento_bert_ta.csv'
batch_texts = textos_metricas['text']
predicted_labels = []

for text in batch_texts:
    predictions = classifier.predict(text)

    # Extract label and score
    label = predictions[0]['label']
    score = predictions[0]['score']

    # Map the label to its corresponding CEFR level using the dictionary
    cefr_label = label_mapping.get(label, label)  # Use the mapping, fallback to original if not found
    predicted_labels.append((cefr_label, score))

    torch.cuda.empty_cache()
    del predictions  # Delete the output after each step to free memory
    gc.collect()


with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Predicted Label', 'Score'])
    for label, score in predicted_labels:
        writer.writerow([label, score])

FileLink(filename)